In [ ]:
import requests
import json
from tqdm import tqdm

PARAMETERS = {
    'Solar8000/HR': 'Heart Rate',
    'Solar8000/ART_MBP': 'Arterial BP',
    'Solar8000/PLETH_SPO2': 'Oxygen Saturation',
    'Solar8000/ETCO2': 'End-Tidal CO2',
    'Solar8000/RR': 'Respiratory Rate',
    'Solar8000/PLETH_HR': 'Pleth HR',
    'Solar8000/ART_SBP': 'Systolic BP',
    'Solar8000/ART_DBP': 'Diastolic BP',
    'Solar8000/NIBP_MBP': 'NIBP Mean BP',
    'Solar8000/CVP': 'Central Venous Pressure',
    'Vigileo/CO': 'Cardiac Output',
    'Vigileo/SV': 'Stroke Volume',
    'Vigilance/HR_AVG': 'Average HR',
    'CardioQ/HR': 'CardioQ HR'
}

CASE_IDS = [25, 58, 68, 92, 96, 116, 135, 137, 142, 161, 203, 229, 239,
            243, 256, 266, 279, 323, 375, 384, 413, 416, 458, 460, 478]

BASE_URL = 'https://api.vitaldb.net'


def interpolate_per_second(raw):
    seconds = {}
    for t, v in raw:
        sec = int(t)
        seconds[sec] = v
    return [{'time': sec, 'value': seconds[sec]} for sec in sorted(seconds)]


result = {}

for case_id in tqdm(CASE_IDS, desc="Cases"):
    try:
        tracks = requests.get(f'{BASE_URL}/cases/{case_id}/trks').json()
    except Exception as e:
        print(f"Error fetching tracks for case {case_id}: {e}")
        continue

    param_tracks = {
        PARAMETERS[trk['name']]: trk['tid']
        for trk in tracks if trk['name'] in PARAMETERS
    }

    case_data = {}
    for label, tid in param_tracks.items():
        try:
            raw = requests.get(f'{BASE_URL}/tracks/{tid}').json()
            values = interpolate_per_second(raw)
            case_data[label] = values
        except Exception as e:
            print(f"Error fetching data for tid {tid}: {e}")
            continue

    result[str(case_id)] = case_data

with open('vital_data.json', 'w') as f:
    json.dump(result, f, indent=2)

print("✅ vital_data.json saved.")




Processing Cases:   0%|          | 0/25 [00:00<?, ?it/s]

  ⏳ Retry: https://api.vitaldb.net/f90f553cf9a9fd18710df990419d677fb17234ab failed (('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None)))


Processing Cases:   0%|          | 0/25 [04:25<?, ?it/s]


KeyboardInterrupt: 